In [68]:
import trompy as tp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
pd.options.display.max_rows= 4000
pd.options.display.max_columns= 4000

np.set_printoptions(threshold=np.inf)  # This sets the threshold to infinity, so the full array is printed


# code below gets all ingestive microstructure including snacks, meals and mega meals, numbers, frequency and size, ipi,imi, daily and hourly data in addition to average/count for all phases, grain, PR and NR.
# here within the temporal threshold of 60 sec, if there is just 1 pellet in the event, it is called snack, 
# if there are between 2 to 4 pellets it is called meal, and if there are more than 4 pellets the behaviour is logged as mega meal or feast.
# in this scenario, meals and mega meals are separate behavioural events,
# if you want to take mega meals as continuation of meals then you should change the meal_metrics function to the code below  




def get_meal_and_snack_metrics(pellettimes, meal_threshold=1/60):
    if not pellettimes:
        return (0, 0, 0, 0, 0, 0, 0, 0, [[0]*24]*7, [[0]*24]*7, [[0]*24]*7, 0, 0, 0)

    IPIs = np.diff(np.array(pellettimes))
    meals = []
    snacks = []
    mega_meals = []
    current_event = [pellettimes[0]]

    # Hourly tracking for each day
    hourly_meals_per_day = [[0]*24 for _ in range(7)]
    hourly_snacks_per_day = [[0]*24 for _ in range(7)]
    hourly_mega_meals_per_day = [[0]*24 for _ in range(7)]

    for i, ipi in enumerate(IPIs):
        if ipi <= meal_threshold:
            current_event.append(pellettimes[i + 1])
        else:
            if len(current_event) == 1:
                snacks.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:  # Make sure we are within the 7-day boundary
                    hourly_snacks_per_day[day_index][hour] += 1
            elif 2 <= len(current_event) <= 4:
                meals.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_meals_per_day[day_index][hour] += 1
            elif len(current_event) >= 5:
                mega_meals.append(current_event)
                meals.append(current_event)  # Count mega_meals as meals too
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_mega_meals_per_day[day_index][hour] += 1
                    hourly_meals_per_day[day_index][hour] += 1  # Count mega_meals in the meal count as well
            current_event = [pellettimes[i + 1]]

    # Handle the last sequence
    if current_event:
        day_index = int(current_event[0] // 24)
        hour = int(current_event[0]) % 24
        if len(current_event) == 1:
            snacks.append(current_event)
            if day_index < 7:
                hourly_snacks_per_day[day_index][hour] += 1
        elif 2 <= len(current_event) <= 4:
            meals.append(current_event)
            if day_index < 7:
                hourly_meals_per_day[day_index][hour] += 1
        elif len(current_event) >= 5:
            mega_meals.append(current_event)
            meals.append(current_event)  # Count mega_meals as meals too
            if day_index < 7:
                hourly_mega_meals_per_day[day_index][hour] += 1
                hourly_meals_per_day[day_index][hour] += 1  # Count mega_meals in the meal count as well

    nmeals = len(meals)
    nsnacks = len(snacks)
    mega_meal_count = len(mega_meals)


In [88]:
# Function to calculate pellets per day
def get_pellets_per_day(timestamps, days=7):
    pellets_per_day = [0] * days
    for day in range(days):
        pellets = [t for t in timestamps if (t > day * 24) and (t < (day + 1) * 24)]
        n_pellets = len(pellets)
        pellets_per_day[day] = n_pellets
    return pellets_per_day

# Function to calculate inter-pellet intervals (IPI)
def get_interpellet_intervals(pellettimes):
    if len(pellettimes) > 1:
        return np.diff(pellettimes)
    return []

# Function to calculate intermeal intervals (IMI)
def get_intermeal_interval(pellettimes, meal_threshold=1/60):
    IPIs = np.diff(pellettimes)
    IMIs = [x for x in IPIs if x > meal_threshold]
    average_IMI = np.mean(IMIs) if IMIs else 0  # Handle case with no IMIs
    return average_IMI

def get_data_subset(dictionary, selectors, verbose=True):
    output_dictionary = dictionary.copy()
    for key, value in selectors.items():
        for mouse_id in dictionary.keys():
            try:
                if output_dictionary[mouse_id][key] != value:
                    output_dictionary.pop(mouse_id)
            except KeyError: 
                pass

    if verbose:
        print("{} items in output dictionary".format(len(output_dictionary.keys())))
    
    return output_dictionary

def get_data_fields(dictionary, fields, selectors):
    output_list = []
    reduced_dictionary = get_data_subset(dictionary, selectors)
    
    if len(reduced_dictionary.keys()) > 0:
        for field in fields:
            output_sublist = []
            try:
                for key in reduced_dictionary.keys():
                    output_sublist.append(reduced_dictionary[key][field])
            except KeyError:
                print("{} is not a key in selected dictionary".format(field))
                return
            output_list.append(output_sublist)
    else:
        print("No data in fields in selected dictionary")

    if len(output_list) == 1:
        output_list = output_list[0]
        
    return output_list
# Function to get meal, snack, and mega meal metrics, including mega_meal_frequency and mega_meal_size
def get_meal_and_snack_metrics(pellettimes, meal_threshold=1/60):
    if not pellettimes:
        return (0, 0, 0, 0, 0, 0, 0, 0, [[0]*24]*7, [[0]*24]*7, [[0]*24]*7, 0, 0, 0, 0)

    IPIs = np.diff(np.array(pellettimes))
    meals = []
    snacks = []
    mega_meals = []
    current_event = [pellettimes[0]]

    # Hourly tracking for each day
    hourly_meals_per_day = [[0]*24 for _ in range(7)]
    hourly_snacks_per_day = [[0]*24 for _ in range(7)]
    hourly_mega_meals_per_day = [[0]*24 for _ in range(7)]

    for i, ipi in enumerate(IPIs):
        if ipi <= meal_threshold:
            current_event.append(pellettimes[i + 1])
        else:
            if len(current_event) == 1:
                snacks.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_snacks_per_day[day_index][hour] += 1
            elif 2 <= len(current_event) <= 4:
                meals.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_meals_per_day[day_index][hour] += 1
            elif len(current_event) >= 5:
                mega_meals.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_mega_meals_per_day[day_index][hour] += 1
            current_event = [pellettimes[i + 1]]

    # Handle the last sequence
    if current_event:
        day_index = int(current_event[0] // 24)
        hour = int(current_event[0]) % 24
        if len(current_event) == 1:
            snacks.append(current_event)
            if day_index < 7:
                hourly_snacks_per_day[day_index][hour] += 1
        elif 2 <= len(current_event) <= 4:
            meals.append(current_event)
            if day_index < 7:
                hourly_meals_per_day[day_index][hour] += 1
        elif len(current_event) >= 5:
            mega_meals.append(current_event)
            if day_index < 7:
                hourly_mega_meals_per_day[day_index][hour] += 1

    nmeals = len(meals)
    nsnacks = len(snacks)
    n_mega_meals = len(mega_meals)

    total_pellets = len(pellettimes)
    mealsize = sum(len(meal) for meal in meals) / nmeals if nmeals else 0
    snack_size = sum(len(snack) for snack in snacks) / nsnacks if nsnacks else 0
    total_observation_period = max(pellettimes) - min(pellettimes)
    meal_frequency = nmeals / total_observation_period if total_observation_period > 0 else 0
    snack_frequency = nsnacks / total_observation_period if total_observation_period > 0 else 0
    mega_meal_frequency = n_mega_meals / total_observation_period if total_observation_period > 0 else 0
    average_mega_meal_size = sum(len(meal) for meal in mega_meals) / n_mega_meals if n_mega_meals else 0

    return (mealsize, snack_size, nmeals, meal_frequency, nsnacks, snack_frequency, 
            mega_meal_frequency, average_mega_meal_size, hourly_meals_per_day, 
            hourly_snacks_per_day, hourly_mega_meals_per_day, meals, snacks, mega_meals, n_mega_meals)


# Load the metafile and process the data
metafile = "..\\FEDProtein_METAFILE.xls"
rows, header = tp.metafilereader(metafile, sheetname="METAFILE")

mice = {}
for row in rows:
    mouse_id = row[1]
    if mouse_id not in mice.keys():
        mice[mouse_id] = {}
        mice[mouse_id]["sex"] = row[4]
        mice[mouse_id]["order"] = row[5]

for key in mice.keys():
    for row in rows:
        if row[1] == key and row[3] == "FF":
            filename = "..\\data\\{}".format(row[0])
            if row[2] == "GRAIN":
                mice[key]["grain_timestamps"] = get_FEDevents(filename, "Pellet")
            elif row[2] == "PR":
                mice[key]["pr_timestamps"] = get_FEDevents(filename, "Pellet")
            elif row[2] == "NR":
                mice[key]["nr_timestamps"] = get_FEDevents(filename, "Pellet")
            else:
                print(row[2], "is not a valid type of pellet for", key)

# Add meal, snack, mega meal metrics, and IMI, IPI calculations to the dictionary
for key in mice.keys():
    grain_timestamps = mice[key].get("grain_timestamps", [])
    pr_timestamps = mice[key].get("pr_timestamps", [])
    nr_timestamps = mice[key].get("nr_timestamps", [])

    # Get metrics for GRAIN, PR, and NR phases
    grain_metrics = get_meal_and_snack_metrics(grain_timestamps)
    pr_metrics = get_meal_and_snack_metrics(pr_timestamps)
    nr_metrics = get_meal_and_snack_metrics(nr_timestamps)

    # GRAIN phase
    (
        mice[key]["grain_meal_size"],
        mice[key]["grain_snack_size"],
        mice[key]["grain_number_of_meals"],
        mice[key]["grain_meal_frequency"],
        mice[key]["grain_number_of_snacks"],
        mice[key]["grain_snack_frequency"],
        mice[key]["grain_mega_meal_frequency"],
        mice[key]["grain_mega_meal_size"],
        mice[key]["grain_hourly_meals"],
        mice[key]["grain_hourly_snacks"],
        mice[key]["grain_hourly_mega_meals"],
        grain_meals,
        grain_snacks,
        grain_mega_meals,
        mice[key]["grain_number_of_mega_meals"]
    ) = grain_metrics

    mice[key]["grain_ipi"] = get_interpellet_intervals(grain_timestamps)  # IPI for GRAIN
    mice[key]["grain_imi"] = get_intermeal_interval(grain_timestamps)  # IMI for GRAIN

    # PR phase
    (
        mice[key]["pr_meal_size"],
        mice[key]["pr_snack_size"],
        mice[key]["pr_number_of_meals"],
        mice[key]["pr_meal_frequency"],
        mice[key]["pr_number_of_snacks"],
        mice[key]["pr_snack_frequency"],
        mice[key]["pr_mega_meal_frequency"],
        mice[key]["pr_mega_meal_size"],
        mice[key]["pr_hourly_meals"],
        mice[key]["pr_hourly_snacks"],
        mice[key]["pr_hourly_mega_meals"],
        pr_meals,
        pr_snacks,
        pr_mega_meals,
        mice[key]["pr_number_of_mega_meals"]
    ) = pr_metrics

    mice[key]["pr_ipi"] = get_interpellet_intervals(pr_timestamps)  # IPI for PR
    mice[key]["pr_imi"] = get_intermeal_interval(pr_timestamps)  # IMI for PR

    # NR phase
    (
        mice[key]["nr_meal_size"],
        mice[key]["nr_snack_size"],
        mice[key]["nr_number_of_meals"],
        mice[key]["nr_meal_frequency"],
        mice[key]["nr_number_of_snacks"],
        mice[key]["nr_snack_frequency"],
        mice[key]["nr_mega_meal_frequency"],
        mice[key]["nr_mega_meal_size"],
        mice[key]["nr_hourly_meals"],
        mice[key]["nr_hourly_snacks"],
        mice[key]["nr_hourly_mega_meals"],
        nr_meals,
        nr_snacks,
        nr_mega_meals,
        mice[key]["nr_number_of_mega_meals"]
    ) = nr_metrics

    mice[key]["nr_ipi"] = get_interpellet_intervals(nr_timestamps)  # IPI for NR
    mice[key]["nr_imi"] = get_intermeal_interval(nr_timestamps)  # IMI for NR

# Now combine events and pellets per day for the entire 17-day time course (as before)
for key in mice.keys():
    mice[key]["all_pellets_per_day"] = (
        mice[key].get("grain_pellets_per_day", [0] * 3) +
        mice[key].get("pr_pellets_per_day", [0] * 7) +
        mice[key].get("nr_pellets_per_day", [0] * 7)
    )

    mice[key]["all_meals_per_day"] = (
        mice[key].get("grain_meals_per_day", [0] * 3) +
        mice[key].get("pr_meals_per_day", [0] * 7) +
        mice[key].get("nr_meals_per_day", [0] * 7)
    )

    mice[key]["all_snacks_per_day"] = (
        mice[key].get("grain_snacks_per_day", [0] * 3) +
        mice[key].get("pr_snacks_per_day", [0] * 7) +
        mice[key].get("nr_snacks_per_day", [0] * 7)
    )

    mice[key]["all_mega_meals_per_day"] = (
        mice[key].get("grain_mega_meals_per_day", [0] * 3) +
        mice[key].get("pr_mega_meals_per_day", [0] * 7) +
        mice[key].get("nr_mega_meals_per_day", [0] * 7)
    )


In [89]:
mice["FEDXB01"].keys()

dict_keys(['sex', 'order', 'grain_timestamps', 'pr_timestamps', 'nr_timestamps', 'grain_meal_size', 'grain_snack_size', 'grain_number_of_meals', 'grain_meal_frequency', 'grain_number_of_snacks', 'grain_snack_frequency', 'grain_mega_meal_frequency', 'grain_mega_meal_size', 'grain_hourly_meals', 'grain_hourly_snacks', 'grain_hourly_mega_meals', 'grain_number_of_mega_meals', 'grain_ipi', 'grain_imi', 'pr_meal_size', 'pr_snack_size', 'pr_number_of_meals', 'pr_meal_frequency', 'pr_number_of_snacks', 'pr_snack_frequency', 'pr_mega_meal_frequency', 'pr_mega_meal_size', 'pr_hourly_meals', 'pr_hourly_snacks', 'pr_hourly_mega_meals', 'pr_number_of_mega_meals', 'pr_ipi', 'pr_imi', 'nr_meal_size', 'nr_snack_size', 'nr_number_of_meals', 'nr_meal_frequency', 'nr_number_of_snacks', 'nr_snack_frequency', 'nr_mega_meal_frequency', 'nr_mega_meal_size', 'nr_hourly_meals', 'nr_hourly_snacks', 'nr_hourly_mega_meals', 'nr_number_of_mega_meals', 'nr_ipi', 'nr_imi', 'all_pellets_per_day', 'all_meals_per_day',

In [90]:
nrpr_male_selector = {"order": 1, "sex": "M"}
prnr_male_selector = {"order": 2, "sex": "M"}
nrpr_female_selector = {"order": 1, "sex": "F"}
prnr_female_selector = {"order": 2, "sex": "F"} 

In [91]:
nrpr_m_mega_meals = get_data_fields(mice, ["all_meals_per_day"], nrpr_male_selector)
prnr_m_mega_meals = get_data_fields(mice, ["all_meals_per_day"], prnr_male_selector)
nrpr_f_mega_meals = get_data_fields(mice, ["all_meals_per_day"], nrpr_female_selector)
prnr_f_mega_meals = get_data_fields(mice, ["all_meals_per_day"], prnr_female_selector)


6 items in output dictionary
6 items in output dictionary
5 items in output dictionary
6 items in output dictionary


In [92]:
nrpr_m_mega_meals

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [10]:
# Function to save results to a CSV file, including all available data, mega_meal_size, IMI, IPI, and Number of Mega Meals
def save_results_to_csv(mice, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header row, including IMI, IPI, and Number of Mega Meals for each phase
        header = [
            'Mouse ID', 'Sex', 'Order', 
            'Grain Meal Size', 'Grain Snack Size', 'Grain Number of Meals', 'Grain Meal Frequency', 
            'Grain Number of Snacks', 'Grain Snack Frequency', 'Grain Mega Meal Frequency', 'Grain Mega Meal Size',
            'Grain Number of Mega Meals',  # Added Grain Number of Mega Meals
            'Grain Hourly Meals', 'Grain Hourly Snacks', 'Grain Hourly Mega Meals',
            'Grain IMI', 'Grain IPI',  # Added IMI and IPI for Grain
            'PR Meal Size', 'PR Snack Size', 'PR Number of Meals', 'PR Meal Frequency', 
            'PR Number of Snacks', 'PR Snack Frequency', 'PR Mega Meal Frequency', 'PR Mega Meal Size',
            'PR Number of Mega Meals',  # Added PR Number of Mega Meals
            'PR Hourly Meals', 'PR Hourly Snacks', 'PR Hourly Mega Meals',
            'PR IMI', 'PR IPI',  # Added IMI and IPI for PR
            'NR Meal Size', 'NR Snack Size', 'NR Number of Meals', 'NR Meal Frequency', 
            'NR Number of Snacks', 'NR Snack Frequency', 'NR Mega Meal Frequency', 'NR Mega Meal Size',
            'NR Number of Mega Meals',  # Added NR Number of Mega Meals
            'NR Hourly Meals', 'NR Hourly Snacks', 'NR Hourly Mega Meals',
            'NR IMI', 'NR IPI',  # Added IMI and IPI for NR
            'All Meals Per Day', 'All Snacks Per Day', 'All Mega Meals Per Day', 'All Pellets Per Day'
        ]
        writer.writerow(header)

        # Write the data rows for each mouse
        for key, mouse_data in mice.items():
            row = [
                key, mouse_data['sex'], mouse_data['order'],
                mouse_data.get('grain_meal_size', 0), mouse_data.get('grain_snack_size', 0),
                mouse_data.get('grain_number_of_meals', 0), mouse_data.get('grain_meal_frequency', 0),
                mouse_data.get('grain_number_of_snacks', 0), mouse_data.get('grain_snack_frequency', 0),
                mouse_data.get('grain_mega_meal_frequency', 0), mouse_data.get('grain_mega_meal_size', 0),  # Added mega_meal_size for GRAIN
                mouse_data.get('grain_number_of_mega_meals', 0),  # Added Number of Mega Meals for GRAIN
                mouse_data.get('grain_hourly_meals', []), mouse_data.get('grain_hourly_snacks', []), mouse_data.get('grain_hourly_mega_meals', []),
                mouse_data.get('grain_imi', 0), mouse_data.get('grain_ipi', []),  # Added IMI and IPI for GRAIN
                mouse_data.get('pr_meal_size', 0), mouse_data.get('pr_snack_size', 0),
                mouse_data.get('pr_number_of_meals', 0), mouse_data.get('pr_meal_frequency', 0),
                mouse_data.get('pr_number_of_snacks', 0), mouse_data.get('pr_snack_frequency', 0),
                mouse_data.get('pr_mega_meal_frequency', 0), mouse_data.get('pr_mega_meal_size', 0),  # Added mega_meal_size for PR
                mouse_data.get('pr_number_of_mega_meals', 0),  # Added Number of Mega Meals for PR
                mouse_data.get('pr_hourly_meals', []), mouse_data.get('pr_hourly_snacks', []), mouse_data.get('pr_hourly_mega_meals', []),
                mouse_data.get('pr_imi', 0), mouse_data.get('pr_ipi', []),  # Added IMI and IPI for PR
                mouse_data.get('nr_meal_size', 0), mouse_data.get('nr_snack_size', 0),
                mouse_data.get('nr_number_of_meals', 0), mouse_data.get('nr_meal_frequency', 0),
                mouse_data.get('nr_number_of_snacks', 0), mouse_data.get('nr_snack_frequency', 0),
                mouse_data.get('nr_mega_meal_frequency', 0), mouse_data.get('nr_mega_meal_size', 0),  # Added mega_meal_size for NR
                mouse_data.get('nr_number_of_mega_meals', 0),  # Added Number of Mega Meals for NR
                mouse_data.get('nr_hourly_meals', []), mouse_data.get('nr_hourly_snacks', []), mouse_data.get('nr_hourly_mega_meals', []),
                mouse_data.get('nr_imi', 0), mouse_data.get('nr_ipi', []),  # Added IMI and IPI for NR
                mouse_data.get('all_meals_per_day', []), mouse_data.get('all_snacks_per_day', []),
                mouse_data.get('all_mega_meals_per_day', []), mouse_data.get('all_pellets_per_day', [])
            ]
            writer.writerow(row)


# Example to save the results to a CSV file
save_results_to_csv(mice, '../results/FEDPROTEIN_UPDATE_V4_results_all_values.csv')


In [11]:
mice["FEDXB01"].keys()

dict_keys(['sex', 'order', 'grain_timestamps', 'pr_timestamps', 'nr_timestamps', 'grain_meal_size', 'grain_snack_size', 'grain_number_of_meals', 'grain_meal_frequency', 'grain_number_of_snacks', 'grain_snack_frequency', 'grain_mega_meal_frequency', 'grain_mega_meal_size', 'grain_hourly_meals', 'grain_hourly_snacks', 'grain_hourly_mega_meals', 'grain_number_of_mega_meals', 'grain_ipi', 'grain_imi', 'pr_meal_size', 'pr_snack_size', 'pr_number_of_meals', 'pr_meal_frequency', 'pr_number_of_snacks', 'pr_snack_frequency', 'pr_mega_meal_frequency', 'pr_mega_meal_size', 'pr_hourly_meals', 'pr_hourly_snacks', 'pr_hourly_mega_meals', 'pr_number_of_mega_meals', 'pr_ipi', 'pr_imi', 'nr_meal_size', 'nr_snack_size', 'nr_number_of_meals', 'nr_meal_frequency', 'nr_number_of_snacks', 'nr_snack_frequency', 'nr_mega_meal_frequency', 'nr_mega_meal_size', 'nr_hourly_meals', 'nr_hourly_snacks', 'nr_hourly_mega_meals', 'nr_number_of_mega_meals', 'nr_ipi', 'nr_imi', 'all_pellets_per_day', 'all_meals_per_day',

# the code failed to properly store the data of imi and ipi and stored all_meals/snacks/pellets/etc per day as a list of 0 , below I am trying to sort it out

# V6 of the code below

In [74]:
import csv
import numpy as np
from datetime import datetime
import trompy as tp

# Function to calculate inter-pellet intervals (IPI)
def get_interpellet_intervals(pellettimes):
    if len(pellettimes) > 1:
        return np.diff(pellettimes)
    return []

# Function to calculate intermeal intervals (IMI)
def get_intermeal_interval(pellettimes, meal_threshold=1/60):
    IPIs = np.diff(pellettimes)
    IMIs = [x for x in IPIs if x > meal_threshold]
    return IMIs

# Function to calculate pellets per day
def get_pellets_per_day(timestamps, days=7):
    pellets_per_day = [0] * days
    for day in range(days):
        pellets = [t for t in timestamps if (t > day * 24) and (t < (day + 1) * 24)]
        n_pellets = len(pellets)
        pellets_per_day[day] = n_pellets
    return pellets_per_day

# Function to calculate events (meals, snacks, mega meals) per day based on timestamps
def get_events_per_day(events, days=7):
    events_per_day = [0] * days  # Initialize with zeros for the duration of the phase
    for event_list in events:
        if len(event_list) > 0:
            event_day = int(event_list[0] // 24)
            if event_day < days:
                events_per_day[event_day] += 1
    return events_per_day


def get_data_subset(dictionary, selectors, verbose=True):
    output_dictionary = dictionary.copy()
    for key, value in selectors.items():
        for mouse_id in dictionary.keys():
            try:
                if output_dictionary[mouse_id][key] != value:
                    output_dictionary.pop(mouse_id)
            except KeyError: 
                pass

    if verbose:
        print("{} items in output dictionary".format(len(output_dictionary.keys())))
    
    return output_dictionary

def get_data_fields(dictionary, fields, selectors):
    output_list = []
    reduced_dictionary = get_data_subset(dictionary, selectors)
    
    if len(reduced_dictionary.keys()) > 0:
        for field in fields:
            output_sublist = []
            try:
                for key in reduced_dictionary.keys():
                    output_sublist.append(reduced_dictionary[key][field])
            except KeyError:
                print("{} is not a key in selected dictionary".format(field))
                return
            output_list.append(output_sublist)
    else:
        print("No data in fields in selected dictionary")

    if len(output_list) == 1:
        output_list = output_list[0]
        
    return output_list


# Function to calculate meal, snack, and mega meal metrics
def get_meal_and_snack_metrics(pellettimes, meal_threshold=1/60):
    if not pellettimes:
        return (0, 0, 0, 0, 0, 0, 0, 0, [[0]*24]*7, [[0]*24]*7, [[0]*24]*7, 0, 0, 0, 0)

    IPIs = np.diff(np.array(pellettimes))
    meals = []
    snacks = []
    mega_meals = []
    current_event = [pellettimes[0]]

    # Hourly tracking for each day
    hourly_meals_per_day = [[0]*24 for _ in range(7)]
    hourly_snacks_per_day = [[0]*24 for _ in range(7)]
    hourly_mega_meals_per_day = [[0]*24 for _ in range(7)]

    for i, ipi in enumerate(IPIs):
        if ipi <= meal_threshold:
            current_event.append(pellettimes[i + 1])
        else:
            if len(current_event) == 1:
                snacks.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_snacks_per_day[day_index][hour] += 1
            elif 2 <= len(current_event) <= 4:
                meals.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_meals_per_day[day_index][hour] += 1
            elif len(current_event) >= 5:
                mega_meals.append(current_event)
                day_index = int(current_event[0] // 24)
                hour = int(current_event[0]) % 24
                if day_index < 7:
                    hourly_mega_meals_per_day[day_index][hour] += 1
            current_event = [pellettimes[i + 1]]

    # Handle the last sequence
    if current_event:
        day_index = int(current_event[0] // 24)
        hour = int(current_event[0]) % 24
        if len(current_event) == 1:
            snacks.append(current_event)
            if day_index < 7:
                hourly_snacks_per_day[day_index][hour] += 1
        elif 2 <= len(current_event) <= 4:
            meals.append(current_event)
            if day_index < 7:
                hourly_meals_per_day[day_index][hour] += 1
        elif len(current_event) >= 5:
            mega_meals.append(current_event)
            if day_index < 7:
                hourly_mega_meals_per_day[day_index][hour] += 1

    nmeals = len(meals)
    nsnacks = len(snacks)
    n_mega_meals = len(mega_meals)

    # Calculate sizes and frequencies
    total_observation_period = max(pellettimes) - min(pellettimes)
    meal_size = sum(len(meal) for meal in meals) / nmeals if nmeals else 0
    snack_size = sum(len(snack) for snack in snacks) / nsnacks if nsnacks else 0
    mega_meal_size = sum(len(meal) for meal in mega_meals) / n_mega_meals if n_mega_meals else 0
    meal_frequency = nmeals / total_observation_period if total_observation_period > 0 else 0
    snack_frequency = nsnacks / total_observation_period if total_observation_period > 0 else 0
    mega_meal_frequency = n_mega_meals / total_observation_period if total_observation_period > 0 else 0

    # Events per day (this part was missing previously)
    meals_per_day = get_events_per_day(meals, days=7)
    snacks_per_day = get_events_per_day(snacks, days=7)
    mega_meals_per_day = get_events_per_day(mega_meals, days=7)

    return (meal_size, snack_size, nmeals, meal_frequency, nsnacks, snack_frequency, 
            mega_meal_frequency, mega_meal_size, hourly_meals_per_day, hourly_snacks_per_day, 
            hourly_mega_meals_per_day, meals_per_day, snacks_per_day, mega_meals_per_day, n_mega_meals)

# Load the metafile and process the data
metafile = "..\\FEDProtein_METAFILE.xls"
rows, header = tp.metafilereader(metafile, sheetname="METAFILE")

mice = {}
for row in rows:
    mouse_id = row[1]
    if mouse_id not in mice.keys():
        mice[mouse_id] = {}
        mice[mouse_id]["sex"] = row[4]
        mice[mouse_id]["order"] = row[5]

for key in mice.keys():
    for row in rows:
        if row[1] == key and row[3] == "FF":
            filename = "..\\data\\{}".format(row[0])
            if row[2] == "GRAIN":
                mice[key]["grain_timestamps"] = get_FEDevents(filename, "Pellet")
            elif row[2] == "PR":
                mice[key]["pr_timestamps"] = get_FEDevents(filename, "Pellet")
            elif row[2] == "NR":
                mice[key]["nr_timestamps"] = get_FEDevents(filename, "Pellet")
            else:
                print(row[2], "is not a valid type of pellet for", key)

# Add meal, snack, mega meal metrics, and IMI, IPI calculations to the dictionary
for key in mice.keys():
    grain_timestamps = mice[key].get("grain_timestamps", [])
    pr_timestamps = mice[key].get("pr_timestamps", [])
    nr_timestamps = mice[key].get("nr_timestamps", [])

    # Get metrics for GRAIN, PR, and NR phases
    grain_metrics = get_meal_and_snack_metrics(grain_timestamps)
    pr_metrics = get_meal_and_snack_metrics(pr_timestamps)
    nr_metrics = get_meal_and_snack_metrics(nr_timestamps)

    # Store GRAIN metrics
    (
        mice[key]["grain_meal_size"],
        mice[key]["grain_snack_size"],
        mice[key]["grain_number_of_meals"],
        mice[key]["grain_meal_frequency"],
        mice[key]["grain_number_of_snacks"],
        mice[key]["grain_snack_frequency"],
        mice[key]["grain_mega_meal_frequency"],
        mice[key]["grain_mega_meal_size"],
        mice[key]["grain_hourly_meals"],
        mice[key]["grain_hourly_snacks"],
        mice[key]["grain_hourly_mega_meals"],
        mice[key]["grain_meals_per_day"],
        mice[key]["grain_snacks_per_day"],
        mice[key]["grain_mega_meals_per_day"],
        mice[key]["grain_number_of_mega_meals"]
    ) = grain_metrics

    # IMI and IPI for GRAIN
    mice[key]["grain_ipi"] = get_interpellet_intervals(grain_timestamps)
    mice[key]["grain_imi"] = get_intermeal_interval(grain_timestamps)

    # Store PR metrics
    (
        mice[key]["pr_meal_size"],
        mice[key]["pr_snack_size"],
        mice[key]["pr_number_of_meals"],
        mice[key]["pr_meal_frequency"],
        mice[key]["pr_number_of_snacks"],
        mice[key]["pr_snack_frequency"],
        mice[key]["pr_mega_meal_frequency"],
        mice[key]["pr_mega_meal_size"],
        mice[key]["pr_hourly_meals"],
        mice[key]["pr_hourly_snacks"],
        mice[key]["pr_hourly_mega_meals"],
        mice[key]["pr_meals_per_day"],
        mice[key]["pr_snacks_per_day"],
        mice[key]["pr_mega_meals_per_day"],
        mice[key]["pr_number_of_mega_meals"]
    ) = pr_metrics

    # IMI and IPI for PR
    mice[key]["pr_ipi"] = get_interpellet_intervals(pr_timestamps)
    mice[key]["pr_imi"] = get_intermeal_interval(pr_timestamps)

    # Store NR metrics
    (
        mice[key]["nr_meal_size"],
        mice[key]["nr_snack_size"],
        mice[key]["nr_number_of_meals"],
        mice[key]["nr_meal_frequency"],
        mice[key]["nr_number_of_snacks"],
        mice[key]["nr_snack_frequency"],
        mice[key]["nr_mega_meal_frequency"],
        mice[key]["nr_mega_meal_size"],
        mice[key]["nr_hourly_meals"],
        mice[key]["nr_hourly_snacks"],
        mice[key]["nr_hourly_mega_meals"],
        mice[key]["nr_meals_per_day"],
        mice[key]["nr_snacks_per_day"],
        mice[key]["nr_mega_meals_per_day"],
        mice[key]["nr_number_of_mega_meals"]
    ) = nr_metrics

    # IMI and IPI for NR
    mice[key]["nr_ipi"] = get_interpellet_intervals(nr_timestamps)
    mice[key]["nr_imi"] = get_intermeal_interval(nr_timestamps)

# Now combine events and pellets per day for the entire 17-day time course
for key in mice.keys():
    # Calculate and combine pellets per day for 17 days (3 days GRAIN + 7 days PR + 7 days NR)
    mice[key]["grain_pellets_per_day"] = get_pellets_per_day(mice[key]["grain_timestamps"], days=3)
    mice[key]["pr_pellets_per_day"] = get_pellets_per_day(mice[key]["pr_timestamps"], days=7)
    mice[key]["nr_pellets_per_day"] = get_pellets_per_day(mice[key]["nr_timestamps"], days=7)

    mice[key]["all_pellets_per_day"] = (
        mice[key]["grain_pellets_per_day"] +
        mice[key]["pr_pellets_per_day"] +
        mice[key]["nr_pellets_per_day"]
    )

    # Combine meals per day
    mice[key]["all_meals_per_day"] = (
        mice[key]["grain_meals_per_day"] +
        mice[key]["pr_meals_per_day"] +
        mice[key]["nr_meals_per_day"]
    )

    # Combine snacks per day
    mice[key]["all_snacks_per_day"] = (
        mice[key]["grain_snacks_per_day"] +
        mice[key]["pr_snacks_per_day"] +
        mice[key]["nr_snacks_per_day"]
    )

    # Combine mega meals per day
    mice[key]["all_mega_meals_per_day"] = (
        mice[key]["grain_mega_meals_per_day"] +
        mice[key]["pr_mega_meals_per_day"] +
        mice[key]["nr_mega_meals_per_day"]
    )

# Example: Output the combined results for one mouse
for key, value in mice.items():
    print(f"Mouse {key}:")
    print("  Pellets per day:", mice[key]["all_pellets_per_day"])
    print("  Meals per day:", mice[key]["all_meals_per_day"])
    print("  Snacks per day:", mice[key]["all_snacks_per_day"])
    print("  Mega Meals per day:", mice[key]["all_mega_meals_per_day"])
    print("  IMI for Grain phase:", mice[key]["grain_imi"])
    print("  IPI for Grain phase:", mice[key]["grain_ipi"])
    print("  IMI for PR phase:", mice[key]["pr_imi"])
    print("  IPI for PR phase:", mice[key]["pr_ipi"])
    print("  IMI for NR phase:", mice[key]["nr_imi"])
    print("  IPI for NR phase:", mice[key]["nr_ipi"])



Mouse FEDXA01:
  Pellets per day: [340, 271, 224, 161, 155, 176, 184, 215, 231, 240, 246, 234, 222, 211, 241, 223, 219]
  Meals per day: [43, 25, 23, 2, 0, 0, 0, 35, 28, 31, 24, 24, 31, 14, 9, 18, 18, 15, 13, 9, 6]
  Snacks per day: [30, 18, 25, 2, 0, 0, 0, 32, 44, 52, 62, 28, 25, 13, 13, 14, 14, 19, 18, 12, 13]
  Mega Meals per day: [25, 22, 18, 0, 0, 0, 0, 6, 7, 8, 9, 19, 19, 28, 27, 24, 23, 22, 25, 22, 25]
  IMI for Grain phase: [0.07138888888888895, 0.0413888888888887, 0.07583333333333364, 0.10111111111111137, 0.025555555555555554, 0.08527777777777779, 0.05638888888888882, 0.13527777777777805, 0.03611111111111143, 0.03833333333333311, 0.0280555555555555, 0.025833333333333375, 0.1466666666666665, 0.07111111111111112, 0.081666666666667, 0.0669444444444447, 0.6952777777777781, 0.06111111111111178, 0.5077777777777772, 0.017777777777777892, 0.06916666666666682, 0.18333333333333357, 0.8022222222222224, 1.2602777777777785, 0.03249999999999975, 1.3744444444444435, 0.026944444444444215, 1.0

In [82]:
# Function to save results to a CSV file, excluding long list data
def save_main_results_to_csv(mice, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header row for the main data
        header = [
            'Mouse ID', 'Sex', 'Order', 
            'Grain Meal Size', 'Grain Snack Size', 'Grain Number of Meals', 'Grain Meal Frequency', 
            'Grain Number of Snacks', 'Grain Snack Frequency', 'Grain Mega Meal Frequency', 'Grain Mega Meal Size', 
            'Grain Number of Mega Meals', 
            'PR Meal Size', 'PR Snack Size', 'PR Number of Meals', 'PR Meal Frequency', 
            'PR Number of Snacks', 'PR Snack Frequency', 'PR Mega Meal Frequency', 'PR Mega Meal Size', 
            'PR Number of Mega Meals', 
            'NR Meal Size', 'NR Snack Size', 'NR Number of Meals', 'NR Meal Frequency', 
            'NR Number of Snacks', 'NR Snack Frequency', 'NR Mega Meal Frequency', 'NR Mega Meal Size', 
            'NR Number of Mega Meals', 
            'All Meals Per Day', 'All Snacks Per Day', 'All Mega Meals Per Day', 'All Pellets Per Day'
        ]
        writer.writerow(header)

        # Write the data rows for each mouse, excluding long list data
        for key, mouse_data in mice.items():
            row = [
                key, mouse_data['sex'], mouse_data['order'],
                mouse_data.get('grain_meal_size', 0), mouse_data.get('grain_snack_size', 0),
                mouse_data.get('grain_number_of_meals', 0), mouse_data.get('grain_meal_frequency', 0),
                mouse_data.get('grain_number_of_snacks', 0), mouse_data.get('grain_snack_frequency', 0),
                mouse_data.get('grain_mega_meal_frequency', 0), mouse_data.get('grain_mega_meal_size', 0),
                mouse_data.get('grain_number_of_mega_meals', 0), 
                mouse_data.get('pr_meal_size', 0), mouse_data.get('pr_snack_size', 0),
                mouse_data.get('pr_number_of_meals', 0), mouse_data.get('pr_meal_frequency', 0),
                mouse_data.get('pr_number_of_snacks', 0), mouse_data.get('pr_snack_frequency', 0),
                mouse_data.get('pr_mega_meal_frequency', 0), mouse_data.get('pr_mega_meal_size', 0),
                mouse_data.get('pr_number_of_mega_meals', 0), 
                mouse_data.get('nr_meal_size', 0), mouse_data.get('nr_snack_size', 0),
                mouse_data.get('nr_number_of_meals', 0), mouse_data.get('nr_meal_frequency', 0),
                mouse_data.get('nr_number_of_snacks', 0), mouse_data.get('nr_snack_frequency', 0),
                mouse_data.get('nr_mega_meal_frequency', 0), mouse_data.get('nr_mega_meal_size', 0),
                mouse_data.get('nr_number_of_mega_meals', 0),
                ', '.join(map(str, mouse_data.get('all_meals_per_day', []))),
                ', '.join(map(str, mouse_data.get('all_snacks_per_day', []))),
                ', '.join(map(str, mouse_data.get('all_mega_meals_per_day', []))),
                ', '.join(map(str, mouse_data.get('all_pellets_per_day', [])))
            ]
            writer.writerow(row)

# Save the main data to CSV
save_main_results_to_csv(mice, '../results/FEDPROTEIN_UPDATE_main_results(SEMI_FINAL1).csv')


In [85]:
# Function to save IMI and IPI data to a separate CSV file
def save_imi_ipi_data_to_csv(mice, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header row for the IMI and IPI data
        header = [
            'Mouse ID', 
            'Grain IMI', 'Grain IPI', 
            'PR IMI', 'PR IPI', 
            'NR IMI', 'NR IPI'
        ]
        writer.writerow(header)

        # Write the IMI and IPI data rows for each mouse
        for key, mouse_data in mice.items():
            row = [
                key,
                ', '.join(map(str, mouse_data.get('grain_imi', []))),
                ', '.join(map(str, mouse_data.get('grain_ipi', []))),
                ', '.join(map(str, mouse_data.get('pr_imi', []))),
                ', '.join(map(str, mouse_data.get('pr_ipi', []))),
                ', '.join(map(str, mouse_data.get('nr_imi', []))),
                ', '.join(map(str, mouse_data.get('nr_ipi', [])))
            ]
            writer.writerow(row)

# Save the IMI and IPI data to CSV
save_imi_ipi_data_to_csv(mice, '../results/FEDPROTEIN_UPDATE_imi_ipi_data.csv')


In [86]:
# Function to save hourly data to a separate CSV file
def save_hourly_data_to_csv(mice, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header row for the hourly data
        header = [
            'Mouse ID', 
            'Grain Hourly Meals', 'Grain Hourly Snacks', 'Grain Hourly Mega Meals',
            'PR Hourly Meals', 'PR Hourly Snacks', 'PR Hourly Mega Meals',
            'NR Hourly Meals', 'NR Hourly Snacks', 'NR Hourly Mega Meals'
        ]
        writer.writerow(header)

        # Write the hourly data rows for each mouse
        for key, mouse_data in mice.items():
            row = [
                key,
                ', '.join(map(str, mouse_data.get('grain_hourly_meals', []))),
                ', '.join(map(str, mouse_data.get('grain_hourly_snacks', []))),
                ', '.join(map(str, mouse_data.get('grain_hourly_mega_meals', []))),
                ', '.join(map(str, mouse_data.get('pr_hourly_meals', []))),
                ', '.join(map(str, mouse_data.get('pr_hourly_snacks', []))),
                ', '.join(map(str, mouse_data.get('pr_hourly_mega_meals', []))),
                ', '.join(map(str, mouse_data.get('nr_hourly_meals', []))),
                ', '.join(map(str, mouse_data.get('nr_hourly_snacks', []))),
                ', '.join(map(str, mouse_data.get('nr_hourly_mega_meals', [])))
            ]
            writer.writerow(row)

# Save the hourly data to CSV
save_hourly_data_to_csv(mice, '../results/FEDPROTEIN_UPDATE_hourly_data.csv')
